![uc3m](img/uc3m.jpg)

# The Rental Car Planning Problem

<a href="http://www.est.uc3m.es/nogales" target="_blank">Javier Nogales</a>



## Summary

Every week, a company must arrange rental cars for the travels of its executives

For the next week, the daily car requirements of the
executives are the following:


|     | Sat | Sun | Mon | Tue | Wed | Thu | Fry |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Requirement | 2 | 5 | 10 | 9 | 16 | 7 | 11 |


A car rental agency offers the following rental prices per car (in euros) for the company:

|  | Price |
| --- | --- |
| Daily cost Sat/Sun | 35 |
| Daily cost weekday | 50 |
| Three-day plan (three consecutive weekdays) | 125 |
| Weekend plan (Sat and Sun) | 60 |
| All-weekdays plan (Mon though Fri) | 180 |
| All-week plan (Sat through Fri) | 200 |

You need to give advice to the company about the optimal car rental for the following week

![uc3m](img/rental.jpg)



    






## Modeling

Add here the variables, objective function and constraints

Xij = # cars using plan j

MIN
35(X11+X12) + 50(X23+...+X27) + 125(X33 + X34 + X35) + 60*X4 + 180*X5 + 200*X6


## Formulation with Pyomo


### The data



In [6]:
### Define here the input (data)

import numpy as np

# enter data as numpy arrays

# Prices for objective function
# order is: x(11), x(12), x(21), ..., x(6)
c = np.array([35, 35, 50, 50, 50, 50, 50, 125, 125, 125, 60, 180, 200])

# Data matrix for constraints
# columns are in the same order as before
# rows go from Sat to Fri 
A = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], 
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
              [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
              [0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1],
              [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1],
              [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1],
              [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1]
             ])

# Requirements
b = np.array([2, 5, 10, 9, 16, 7, 11])

# set of row indices
I = range(len(A))

# set of column indices
J = range(len(A.T))

In [11]:
### Write here the code

from pyomo.environ import *

model = ConcreteModel()

model.x = Var(J, within = NonNegativeReals) # J: 13 Within: X>= 0

model.objective = Objective(expr = sum(model.x[j] * c[j] for j in J)) # Sum product

model.constraints = ConstraintList() # A * x >= b
for i in I:
    model.constraints.add(sum(A[i,j]*model.x[j] for j in J) >= b[i]) # pyomo does not admit numpy inside

In [12]:
# import pyomo.environ as pyo
# from pyomo.opt import SolverFactory

In [13]:
# import pyomo.environ as pyo
# from pyomo.opt import SolverFactory
# model = pyo.ConcreteModel()
# model.nVars = pyo.Param(initialize=4)
# model.N = pyo.RangeSet(model.nVars)
# model.x = pyo.Var(model.N, within=pyo.Binary)
# model.obj = pyo.Objective(expr=pyo.summation(model.x))
# model.cuts = pyo.ConstraintList()
# opt = SolverFactory('glpk')
# opt.solve(model) 

In [14]:
### Solve the problem

# Define the solver
opt = SolverFactory('glpk')

# to include marginal prices
model.dual = Suffix(direction=Suffix.IMPORT) 

# Solve
results = opt.solve(model)  # solve the model with the selected solver
#opt.solve(model)

model.display()

    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

### Interpretation


Write here the interpretation (the optimal cost is 2210 eur)

Questions:

- What would change if your estimated requirement for Friday would increase by one?

- And if that unit increase would affect the requirement on Saturday?

- Which day of the week is the most sensitive to a unit increase in the requirement?

